# Paso 1: Data

In [6]:
# Primer paso:
# Lectura de datos
import pandas as pd
import warnings
warnings.filterwarnings("ignore")  # Ignore all warnings
pd.options.display.float_format = '{:,.2f}'.format
import altair as alt

In [7]:
df_p = pd.read_excel('Datos Minería.xlsx', sheet_name='data x p')

In [8]:
df_p.head()

,País,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,CHILE,5.75,5.76,5.55,5.50,5.83,5.79,5.73,5.62,5.33,5.00
1,PERÚ,1.38,1.70,2.35,2.45,2.44,2.46,2.15,2.33,2.45,2.76
2,CONGO - KINSHASA,1.03,1.02,0.85,1.09,1.23,1.29,1.60,1.74,2.35,2.50
3,CHINA,1.76,1.71,1.90,1.71,1.59,1.68,1.72,1.91,1.94,1.70
4,ESTADOS UNIDOS,1.36,1.38,1.43,1.26,1.22,1.26,1.20,1.23,1.23,1.10


In [9]:
df_p_melted = pd.melt(df_p, id_vars=['País'], var_name='Año', value_name='PRODUCCIÓN EN MILLONES (TMF)')
df_p_melted.head()

,País,Año,PRODUCCIÓN EN MILLONES (TMF)
0,CHILE,2014,5.75
1,PERÚ,2014,1.38
2,CONGO - KINSHASA,2014,1.03
3,CHINA,2014,1.76
4,ESTADOS UNIDOS,2014,1.36


# Paso 2: Data to Information
1. ¿Quién es mi audiencia?
2. ¿Qué información necesito?
3. ¿Es la data relevante para responder mi pregunta?

In [10]:
alt.Chart(df_p_melted).mark_line().encode(
    #x = 'Año:O',
    x = alt.X('Año:O', axis = alt.Axis(labelAngle=0)),
    y = 'PRODUCCIÓN EN MILLONES (TMF):Q',
    color = 'País:N'
).properties(
    width=600,
    height= 300
)

alt.Chart(...)

In [11]:
df_p_melted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 3 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   País                          110 non-null    object 
 1   Año                           110 non-null    object 
 2   PRODUCCIÓN EN MILLONES (TMF)  106 non-null    float64
dtypes: float64(1), object(2)
memory usage: 2.7+ KB


In [12]:
df_p_melted['Valor_Rellenado'] = df_p_melted['PRODUCCIÓN EN MILLONES (TMF)'].fillna(df_p_melted['PRODUCCIÓN EN MILLONES (TMF)'].rolling(window=3, min_periods=1).mean())

In [13]:
alt.Chart(df_p_melted).mark_line().encode(
    #x = 'Año:O',
    x = alt.X('Año:O', axis = alt.Axis(labelAngle=0)),
    y = 'Valor_Rellenado:Q',
    color = 'País:N'
).properties(
    width=600,
    height= 300
)

alt.Chart(...)

In [ ]:
df_p_melted.head()

In [14]:
for pais in df_p_melted['País'].unique():
    current = df_p_melted[df_p_melted['País'] == pais]['Valor_Rellenado']
    base = df_p_melted[(df_p_melted['País'] == pais) & (df_p_melted['Año'] == 2014)]['Valor_Rellenado'].values[0]
    df_p_melted.loc[df_p_melted['País'] == pais, 'PI'] = (current - base)/ base*100

In [16]:
df_p_melted.tail()

,País,Año,PRODUCCIÓN EN MILLONES (TMF),Valor_Rellenado,PI
105,INDONESIA,2023,0.84,0.84,-20.08
106,AUSTRALIA,2023,0.81,0.81,-16.49
107,ZAMBIA,2023,0.76,0.76,7.34
108,MÉXICO,2023,0.75,0.75,45.63
109,OTROS,2023,4.58,4.58,6.61


In [17]:
alt.Chart(df_p_melted).mark_line().encode(
    #x = 'Año:O',
    x = alt.X('Año:O', axis = alt.Axis(labelAngle=0)),
    y = 'PI:Q',
    color = 'País:N'
).properties(
    width=600,
    height= 300
)

alt.Chart(...)

In [18]:
base = alt.Chart(df_p_melted).encode(
    #x = 'Año:O',
    x = alt.X('Año:O', axis = alt.Axis(labelAngle=0),title = 'Año'),
    y = alt.Y('PI:Q', title = 'Crecimiento Porcentual desde 2014'),
    color = alt.Color('País:N',
                      scale=alt.Scale(scheme='set2'),
                      legend=None
                     ),
    strokeWidth=alt.condition(alt.datum.País == 'PERÚ', alt.value(7), alt.value(1))

).properties(
    title='Producción Mundial de Cobre (Millones TMF)',
    width=600,
    height= 300
)
anotation = base.mark_text(
    dx=10,
    align='left',
    fontSize=12
).encode(
    text = 'label:N'
).transform_filter(
    alt.datum.Año == 2023
).transform_calculate(
    label='datum.País + "(" + format(datum.PI, ".2f") + "%)"'
)
base.mark_line() + anotation

alt.LayerChart(...)

# Paso 3: Information to Knowledge
1. Agregar contexto (circunstancias, eventos, situaciones, entre otros detalles)
2. Hace más fácil entender los datos. Puedes utilizar títulos, leyendas, imágenes entre otros

In [19]:
df_dept = pd.read_excel('Datos Minería.xlsx', sheet_name='data x dept')
df_dept.head()

,Departamento,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,MOQUEGUA,"183,140.95","183,117.39","174,918.37","161,118.51","162,794.51","158,587.91","170,627.13","172,594.14","244,393.58","471,964.53"
1,AREQUIPA,"236,540.46","256,849.88","523,255.86","502,824.94","496,346.13","478,337.84","392,289.99","422,575.37","463,474.61","470,754.58"
2,ÁNCASH,"371,464.71","422,257.41","454,447.18","447,232.35","467,756.72","466,105.55","399,790.88","464,908.51","472,914.32","443,611.36"
3,APURÍMAC,0.00,"6,666.79","329,368.44","452,949.58","385,308.14","382,524.17","312,776.13","290,106.01","254,838.03","302,039.03"
4,CUSCO,"167,247.02","309,423.41","354,838.09","328,274.22","327,592.74","311,538.81","262,661.54","248,885.18","240,639.69","273,525.40"


In [20]:
df_dept_melted = pd.melt(df_dept, id_vars=['Departamento'], var_name='Año', value_name='PRODUCCIÓN (TMF)')
df_dept_melted

,Departamento,Año,PRODUCCIÓN (TMF)
0,MOQUEGUA,2014,"183,140.95"
1,AREQUIPA,2014,"236,540.46"
2,ÁNCASH,2014,"371,464.71"
3,APURÍMAC,2014,0.00
4,CUSCO,2014,"167,247.02"
...,...,...,...
155,HUANCAVELICA,2023,"9,739.03"
156,HUÁNUCO,2023,"1,852.46"
157,PUNO,2023,"1,071.74"
158,AYACUCHO,2023,551.66


In [21]:
chart = alt.Chart(df_dept_melted).mark_line(point=True).encode(
    x='Año:O',  # Eje X con los años (ordinal)
    y='PRODUCCIÓN (TMF):Q',  # Eje Y con la producción
    color='Departamento:N',  # Colorear las líneas por departamento
    tooltip=['Departamento', 'Año', 'PRODUCCIÓN (TMF)']  # Agregar tooltip para interacción
).properties(
    title='Evolutivo de la Producción de Cobre por Departamento',
    width=600,
    height=400
)

chart.show()

alt.Chart(...)

In [22]:
chart = alt.Chart(df_dept_melted[df_dept_melted.Departamento == 'MOQUEGUA']).mark_line(point=True).encode(
    x = alt.X('Año:O', axis = alt.Axis(labelAngle=0),title = 'Año'),
    y='PRODUCCIÓN (TMF):Q',  # Eje Y con la producción
    color='Departamento:N',  # Colorear las líneas por departamento
    tooltip=['Departamento', 'Año', 'PRODUCCIÓN (TMF)']  # Agregar tooltip para interacción
).properties(
    title='Evolutivo de la Producción de Cobre por Departamento',
    width=600,
    height=400
)

chart.show()

alt.Chart(...)

In [23]:
base = alt.Chart(df_p_melted).encode(
    #x = 'Año:O',
    x = alt.X('Año:O', axis = alt.Axis(labelAngle=0),title = 'Año'),
    y = alt.Y('PI:Q', title = 'Crecimiento Porcentual desde 2014'),
    color = alt.Color('País:N',
                      scale=alt.Scale(scheme='set2'),
                      legend=None
                     ),
    strokeWidth=alt.condition(alt.datum.País == 'PERÚ', alt.value(7), alt.value(1))

).properties(
    title='Producción Mundial de Cobre (Millones TMF)',
    width=600,
    height= 450
)
anotation = base.mark_text(
    dx=10,
    align='left',
    fontSize=12
).encode(
    text = 'label:N'
).transform_filter(
    alt.datum.Año == 2023
).transform_calculate(
    label='datum.País + "(" + format(datum.PI, ".2f") + "%)"'
)

###############



####

mequegua = alt.Chart(df_dept_melted[df_dept_melted.Departamento == 'MOQUEGUA']).mark_line(point=True).encode(
    x = alt.X('Año:O', axis = alt.Axis(labelAngle=0),title = 'Año'),
    y='PRODUCCIÓN (TMF):Q',  # Eje Y con la producción
    color='Departamento:N',  # Colorear las líneas por departamento
    tooltip=['Departamento', 'Año', 'PRODUCCIÓN (TMF)']  # Agregar tooltip para interacción
).properties(
    title='Evolutivo de la Producción Moquegua',
    width=260,
    height=150
)

######

text_comm = f"""Parte del crecimiento 
registrado obedece a la contribución 
del proyecto Quellaveco con 
exportación de cobre, 
que inició sus
operaciones en la segunda
mitad del año 2022 
en la región de Moquegua"""

df_commentary = pd.DataFrame([{'text' : text_comm}])
commentary = alt.Chart(df_commentary).mark_text(
    lineBreak='\n',
    align='left',
    fontSize=15,
    y=50,
    x=0,
    color='#81c01e'
).encode(
    text='text:N'
).properties(
    width=200,
    height=200
)

(base.mark_line() + anotation)|(commentary & mequegua)


alt.HConcatChart(...)

# Paso 4: Knowledge to Wisdom
1. Invitamos a accionar

In [24]:
base = alt.Chart(df_p_melted).encode(
    #x = 'Año:O',
    x = alt.X('Año:O', axis = alt.Axis(labelAngle=0),title = 'Año'),
    y = alt.Y('PI:Q', title = 'Crecimiento Porcentual desde 2014'),
    color = alt.Color('País:N',
                      scale=alt.Scale(scheme='set2'),
                      legend=None
                     ),
    strokeWidth=alt.condition(alt.datum.País == 'PERÚ', alt.value(7), alt.value(1))

).properties(
    title='Producción Mundial de Cobre (Millones TMF)',
    width=600,
    height= 450
)
anotation = base.mark_text(
    dx=10,
    align='left',
    fontSize=12
).encode(
    text = 'label:N'
).transform_filter(
    alt.datum.Año == 2023
).transform_calculate(
    label='datum.País + "(" + format(datum.PI, ".2f") + "%)"'
)

####################################

mequegua = alt.Chart(df_dept_melted[df_dept_melted.Departamento == 'MOQUEGUA']).mark_line(point=True).encode(
    x = alt.X('Año:O', axis = alt.Axis(labelAngle=0),title = 'Año'),
    y='PRODUCCIÓN (TMF):Q',  # Eje Y con la producción
    color='Departamento:N',  # Colorear las líneas por departamento
    tooltip=['Departamento', 'Año', 'PRODUCCIÓN (TMF)']  # Agregar tooltip para interacción
).properties(
    title='Evolutivo de la Producción Moquegua',
    width=300,
    height=200
)

######

text_comm = f"""Parte del crecimiento 
registrado obedece a la contribución 
del proyecto Quellaveco con 
exportación de cobre, 
que inició sus operaciones en la segunda
mitad del año 2022 
en la región de Moquegua"""

df_commentary = pd.DataFrame([{'text' : text_comm}])
commentary = alt.Chart(df_commentary).mark_text(
    lineBreak='\n',
    align='left',
    fontSize=15,
    y=50,
    x=0,
    color='gray'
).encode(
    text='text:N'
).properties(
    width=300,
    height=200
)
############################

text_cta = f"""En vista el gran crecimiento en la producción de cobre deberías 
considerar invertir en Perú!!!"""
df_cta = pd.DataFrame([{'text' : text_cta}])
cta = alt.Chart(df_cta
).mark_text(
lineBreak='\n',
align='left',
fontSize=20,
y=0,
x=0,
color='red'
).encode(
text='text:N'
).properties(
width=200
)


( cta & (base.mark_line() + anotation))|(commentary & mequegua)


alt.HConcatChart(...)

# FIN